Use this utlity to update investment-options.csv

See https://www.myfrs.com/InvestmentFundTabs.htm

Values as of Jan 29, 2023

In [19]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [11]:
import csv
from datetime import datetime
from shutil import copyfile

import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import qgrid

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [12]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [13]:
filename = 'investment-options.csv'

In [14]:
# Read in investment options input csv.
header = None
with open('investment-options-header.csv', 'r') as f:
    header = f.read() + '\n'
df = pd.read_csv(filename, skip_blank_lines=True, comment='#')
symbols = list(df['Investment Option'])

In [15]:
# Get list of all asset classes.
filepath = bb.ROOT / 'universe' / 'asset-class-galaxy' / 'asset-classes.csv'
asset_classes = pd.read_csv(filepath, skip_blank_lines=True, comment='#')
asset_classes = list(asset_classes['Asset Class A'])                
asset_classes = set(asset_classes)
asset_classes = sorted(asset_classes)

In [16]:
# Create qgrid widget.
df['Asset Class'] = pd.Categorical(df['Asset Class'], asset_classes)
column_definitions = {
    'Investment Option': {
        'width': 800,
    },
    'Description': {
        'width': 800,
    },
    'Asset Class': {
        # SlickGrid column options.
        'defaultSortAsc': True,
        'maxWidth': None,
        'minWidth': 50,
        'resizable': True,
        'sortable': True,
        'toolTip': "",
        'width': 800,
        # Qgrid column options.
        'editable': True,
    }
}

qgrid_widget = qgrid.show_grid(df, show_toolbar=True, column_definitions=column_definitions)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [22]:
# Update df_orig with user changes, YOU MUST RUN THIS CELL AFTER MAKING CHANGES.
df = qgrid_widget.get_changed_df()

In [23]:
# Create save button.
def on_button_clicked(_):
    # backup current file
    timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    backup = filename + '-' + timestamp
    copyfile(filename, backup)
    
    # Save changes.
    with open(filename, 'w') as f:
        f.write(header)
    df.to_csv(filename, mode='a', index=False, header=False, quoting=csv.QUOTE_ALL)
    print('Saved changes to "{}"'.format(filename))

# Creates a button that allows us to save our changes.
button = widgets.Button(description='SAVE')
out = widgets.Output()
    
# Linking button and function together using a button's method.
button.on_click(on_button_clicked)

# Displaying button and its output together.
widgets.VBox([button,out])

Saved changes to "investment-options.csv"
